# Introduction

# Prepare

### Import nessesary libraries

In [127]:
import numpy as np

In [149]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.utils as vutils
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm

### Hyper Parameter Configuration

In [225]:
class Config:
    image_size = 128                                                         # Size of images (128x128 or 256x256)
    batch_size = 32
    epochs = 50
    lr = 0.00005                                                              # Learning rate (standard for GANs)
    beta1 = 0.5                                                              # Beta1 hyperparam for Adam optimizers
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    data_path = "./textile_dataset/"                                         # FOLDER WHERE YOU PUT YOUR IMAGES
    output_path = "./generated_patterns"

os.makedirs(Config.output_path, exist_ok=True)
os.makedirs(Config.data_path, exist_ok=True)

print(f"Running on device: {Config.device}")

Running on device: cpu


## Dataset

### Data Loader Model

In [226]:
class TextileDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(root_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        if len(self.image_files) == 0:
            print(f"WARNING: No images found in {root_dir}. Please add carpet/cloth images.")
            # We create dummy data if folder is empty so code doesn't crash
            self.dummy = True
        else:
            self.dummy = False

    def __len__(self):
        return len(self.image_files) if self.image_files else 100 # Return 100 if dummy

    def __getitem__(self, idx):
        if self.dummy:
            # Create a synthetic texture (noise) if no data provided
            return torch.randn(3, Config.image_size, Config.image_size)
            
        img_name = os.path.join(self.root_dir, self.image_files[idx % len(self.image_files)])
        image = Image.open(img_name).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        return image

### Transform

In [227]:
transform = transforms.Compose([
    transforms.Resize((Config.image_size, Config.image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # Normalize to [-1, 1]
])

### Loading

In [228]:
dataset = TextileDataset(Config.data_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=Config.batch_size, shuffle=True, num_workers=0)

In [229]:
len(dataset)

2068

# Fractal Model

## Fractal Generator Model

In [230]:
class FractalEngine:
    """
    Generates mathematical fractals to serve as the structural condition
    for the generative model.
    """
    @staticmethod
    def generate_julia(width, height, c_real, c_imag, zoom=1.0):
        # Create a grid of complex numbers
        x = np.linspace(-1.5/zoom, 1.5/zoom, width)
        y = np.linspace(-1.5/zoom, 1.5/zoom, height)
        X, Y = np.meshgrid(x, y)
        Z = X + 1j * Y
        C = complex(c_real, c_imag)
        
        img = np.zeros(Z.shape, dtype=float)
        mask = np.ones(Z.shape, dtype=bool)
        
        # Iteration count (Simulating fractal depth)
        max_iter = 30
        for i in range(max_iter):
            Z[mask] = Z[mask] * Z[mask] + C
            mask = (np.abs(Z) < 10)
            img[mask] = i
            
        # Normalize to 0-1
        img = img / max_iter
        return torch.tensor(img, dtype=torch.float32).unsqueeze(0) # (1, H, W)

    @staticmethod
    def get_random_batch(batch_size, size):
        """Generates a batch of random fractals"""
        fractals = []
        for _ in range(batch_size):
            # Randomize fractal parameters
            c_re = random.uniform(-0.8, 0.0)
            c_im = random.uniform(0.0, 0.7)
            zoom = random.uniform(0.8, 1.2)
            
            f = FractalEngine.generate_julia(size, size, c_re, c_im, zoom)
            fractals.append(f)
        return torch.stack(fractals).to(Config.device)

# Main Model Design

## Model Architecture

(Pix2Pix Style)

### Generator

Generator: U-Net (Encoder-Decoder with Skip Connections)

Input: 1-channel Fractal -> Output: 3-channel Textile

In [231]:
class UNetGenerator(nn.Module):
    def __init__(self):
        super(UNetGenerator, self).__init__()
        
        def down_block(in_feat, out_feat, normalize=True):
            layers = [nn.Conv2d(in_feat, out_feat, 4, 2, 1, bias=False)]
            if normalize: layers.append(nn.BatchNorm2d(out_feat))
            layers.append(nn.LeakyReLU(0.2))
            return nn.Sequential(*layers)

        def up_block(in_feat, out_feat, dropout=0.0):
            layers = [
                nn.ConvTranspose2d(in_feat, out_feat, 4, 2, 1, bias=False),
                nn.BatchNorm2d(out_feat),
                nn.ReLU(inplace=True)
            ]
            if dropout: layers.append(nn.Dropout(dropout))
            return nn.Sequential(*layers)

        # Encoder
        self.down1 = down_block(1, 64, normalize=False) # 128 -> 64
        self.down2 = down_block(64, 128)               # 64 -> 32
        self.down3 = down_block(128, 256)              # 32 -> 16
        self.down4 = down_block(256, 512)              # 16 -> 8
        self.down5 = down_block(512, 512)              # 8 -> 4
        
        # Decoder (with Skip Connections inputs calculated in forward)
        self.up1 = up_block(512, 512, dropout=0.5)     # 4 -> 8
        self.up2 = up_block(1024, 256, dropout=0.5)    # 8 -> 16 (input is cat(up1, down4))
        self.up3 = up_block(512, 128)                  # 16 -> 32
        self.up4 = up_block(256, 64)                   # 32 -> 64
        
        self.final = nn.Sequential(
            nn.ConvTranspose2d(128, 3, 4, 2, 1),       # 64 -> 128
            nn.Tanh() # Output -1 to 1
        )

    def forward(self, x):
        # Down
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        
        # Up (Concatenate with skip connections)
        u1 = self.up1(d5)
        u2 = self.up2(torch.cat([u1, d4], 1))
        u3 = self.up3(torch.cat([u2, d3], 1))
        u4 = self.up4(torch.cat([u3, d2], 1))
        
        out = self.final(torch.cat([u4, d1], 1))
        return out

### Discriminator

PatchGAN

Input: 3-channel Image -> Output: 1-channel Real/Fake Grid

In [232]:
class PatchDiscriminator(nn.Module):
    def __init__(self):
        super(PatchDiscriminator, self).__init__()
        
        def discriminator_block(in_filters, out_filters, normalization=True):
            layers = [nn.Conv2d(in_filters, out_filters, 4, 2, 1)]
            if normalization:
                layers.append(nn.BatchNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return nn.Sequential(*layers)

        self.model = nn.Sequential(
            discriminator_block(3, 64, normalization=False),
            discriminator_block(64, 128),
            discriminator_block(128, 256),
            discriminator_block(256, 512),
            nn.Conv2d(512, 1, 4, 1, 0),
            nn.Sigmoid()
        )

    def forward(self, img):
        return self.model(img)

# Trainning

## Innitializing

### Models

In [233]:
generator = UNetGenerator().to(Config.device)
discriminator = PatchDiscriminator().to(Config.device)

### Loss functions

> Standard GAN Loss

In [234]:
criterion_GAN = nn.BCELoss()

### Optimizers

In [235]:
optimizer_G = optim.Adam(generator.parameters(), lr=Config.lr, betas=(Config.beta1, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=Config.lr, betas=(Config.beta1, 0.999))

### Weights init

In [236]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)

In [237]:
generator.apply(weights_init)
discriminator.apply(weights_init)

PatchDiscriminator(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (2): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (3): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (4): Conv2d(512, 1, kernel_size=(4, 4

## Tranning Loop

In [238]:
print("Starting Training Loop...")

for epoch in range(1, Config.epochs + 1):
    loop = tqdm(dataloader, leave=True)
    
    for i, real_imgs in enumerate(loop):
        batch_curr_size = real_imgs.size(0)
        real_imgs = real_imgs.to(Config.device)
        
        # 1. Generate Fractal Conditions
        fractal_condition = FractalEngine.get_random_batch(batch_curr_size, Config.image_size)
        
        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()
        
        # Real Images
        # Note: In standard Pix2Pix we pair inputs. Here we use Unpaired training 
        # (similar to Texture Synthesis). We want Real to be classified as Real.
        label_real = torch.ones(batch_curr_size, 1, 4, 4).to(Config.device) * 0.9 # Label smoothing
        output_real = discriminator(real_imgs)
        # PatchGAN outputs a grid, we resize label to match
        label_real = torch.ones_like(output_real).to(Config.device)
        loss_D_real = criterion_GAN(output_real, label_real)
        
        # Fake Images
        fake_imgs = generator(fractal_condition)
        label_fake = torch.zeros_like(output_real).to(Config.device)
        output_fake = discriminator(fake_imgs.detach()) # Detach to avoid G gradients
        loss_D_fake = criterion_GAN(output_fake, label_fake)
        
        loss_D = (loss_D_real + loss_D_fake) * 0.5
        loss_D.backward()
        optimizer_D.step()
        
        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()
        
        # Generator wants Discriminator to think images are Real
        output_fake_for_G = discriminator(fake_imgs)
        label_real_for_G = torch.ones_like(output_fake_for_G).to(Config.device)
        
        # Structural Loss (Optional but good): 
        # Enforce that the output intensity roughly matches fractal intensity 
        # to preserve the pattern shape.
        fake_gray = torch.mean(fake_imgs, dim=1, keepdim=True)
        # Normalize fractal to -1 to 1 for comparison
        fractal_norm = (fractal_condition - 0.5) / 0.5 
        loss_structure = torch.mean(torch.abs(fake_gray - fractal_norm)) * 10 
        
        loss_G_GAN = criterion_GAN(output_fake_for_G, label_real_for_G)
        loss_G = loss_G_GAN + loss_structure
        
        loss_G.backward()
        optimizer_G.step()
        
        # Update progress bar
        loop.set_description(f"Epoch [{epoch}/{Config.epochs}]")
        loop.set_postfix(D_loss=loss_D.item(), G_loss=loss_G.item())
        
    # Save Generated Images every few epochs
    if epoch % 5 == 0:
        with torch.no_grad():
            # Generate a consistent visualization
            # 1. Fractal Input (Grayscale)
            # 2. Generated Textile (RGB)
            viz_fractal = fractal_condition[:4].repeat(1,3,1,1) # Make 3 channel for stacking
            viz_gen = fake_imgs[:4]
            
            # Denormalize for saving
            viz_gen = (viz_gen * 0.5) + 0.5
            
            # Stack vertically: Top row = Fractals, Bottom row = Patterns
            grid = torch.cat([viz_fractal, viz_gen], dim=0)
            vutils.save_image(grid, f"{Config.output_path}/epoch_{epoch}.png", nrow=4)
            
            # Save Model
            torch.save(generator.state_dict(), f"{Config.output_path}/generator.pth")

print("Training Finished!")

Starting Training Loop...


Epoch [50/50]: 100%|█| 65/65 [02:24<00:00,  2.22s/it, D_loss=0.604, G_loss=7.41]

Training Finished!


# Infrence

## Input and Output definning

In [239]:
def generate_design(save_name="final_design.png"):
    generator.eval()
    with torch.no_grad():
        # User defines a specific fractal parameter for their art
        # e.g., A nice Julia set
        f = FractalEngine.generate_julia(Config.image_size, Config.image_size, -0.7, 0.27015, zoom=1.0)
        f = f.unsqueeze(0).to(Config.device)
        
        out = generator(f)
        out = (out * 0.5) + 0.5 # Denormalize
        
        vutils.save_image(out, save_name)
        print(f"Design saved to {save_name}")

## Generate

In [240]:
generate_design()

Design saved to final_design.png
